# Carga de librerias

In [1]:
# • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ Pyfunctions ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ • ~ •
# Librerías y/o depedencias
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from scipy import stats
#from yellowbrick.classifier import ClassBalance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
#import empiricaldist
sns.set_theme(context='notebook', style=plt.style.use('dark_background'))

In [2]:
sys.path.append('pyfunctions.py')

# Carga de datos


In [3]:
df = pd.read_excel('Caso 01 - Cobranza Sector Financiero.xlsx')

# Preprocesamiento y limpieza

In [4]:
predictors = [str(col).lower().strip() for col in df.columns]
df= df.set_axis(axis=1, labels=predictors)
df.head(10)

,consecutivo documento deudor,nit,nombres,numero documento,tipo de producto,producto,valor obligacion,valor vencido,regional,region,...,red,cuadrante,descripcion causal,calificacion real,fecha de perfeccionamiento,endeudamiento,rango endeudamiento,sector economico,profesion,ocupacion
0,28768192,18,NaN,NaN,3,TARJETAS DE CREDITO,5637605.0,152093.0,2099,BOGOTA,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD REDUCIDA,CALAMIDAD,2,1978-12-06,5637605.0,2015-01-01 00:00:00,NaN,SIN PROFESION,SIN OFICIO
1,2712044,22,NaN,NaN,5,CONSUMO,40584.0,40584.0,2099,BOGOTA,...,1,SIN CUADRANTE,SIN CAUSAL DE MORA,3,2015-03-11,40584.0,<1M,NaN,SIN PROFESION,SIN OFICIO
2,13587837,23,NaN,NaN,3,TARJETAS DE CREDITO,2786178.0,547088.0,3099,SUR,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD REDUCIDA,RETRASO EN PAGO DE SALARIO,1,2003-03-27,7747280.0,2015-01-01 00:00:00,NaN,SIN PROFESION,JUBILADO
3,23479925,46,NaN,NaN,3,TARJETAS DE CREDITO,5904653.0,1455631.0,2099,BOGOTA,...,1,NO TIENE VOLUNTAD Y TIENE CAPACIDAD,DESCUIDO,1,1990-09-01,5904653.0,2015-01-01 00:00:00,NaN,SIN PROFESION,SIN OFICIO
4,2712382,92,NaN,NaN,5,CONSUMO,2221125.0,2221125.0,6399,CENTRO,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD,DESCUIDO,2,2015-05-12,2221125.0,2015-01-01 00:00:00,AGROPECUARIO,AGRICULTOR,TAXISTA
5,1683416,104,NaN,NaN,3,TARJETAS DE CREDITO,610840.0,349729.0,4599,CARIBE,...,1,NO TIENE VOLUNTAD Y TIENE CAPACIDAD,INGRESOS INSUFICIENTES,0,1972-10-01,610840.0,<1M,MADERAS Y SUS PRODUCTOS,CONTADOR,SIN OFICIO
6,4270407,106,NaN,NaN,3,TARJETAS DE CREDITO,1527464.0,1527464.0,1099,ANTIOQUIA,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD,DESCUIDO,1,2004-12-03,1574242.0,2015-01-01 00:00:00,TEXTILES Y CONFECCIONES,ADMINISTRADOR,SIN OFICIO
7,21947686,119,NaN,NaN,3,TARJETAS DE CREDITO,4310924.0,1239257.0,2099,BOGOTA,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD,DESCUIDO,1,1998-08-27,4310924.0,2015-01-01 00:00:00,NaN,SIN PROFESION,SIN OFICIO
8,2712477,120,NaN,NaN,5,CONSUMO,1110042.0,1110042.0,6199,CENTRO,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD,INGRESOS INSUFICIENTES,1,2015-06-10,8878707.0,2015-01-01 00:00:00,"CONSTRUCCION, CEMENTO Y ACTIVIDADES INMOBILIARIAS",SIN PROFESION,INDEPENDIENTE
9,19927288,129,NaN,NaN,3,TARJETAS DE CREDITO,5228504.0,547011.0,3099,SUR,...,1,TIENE VOLUNTAD Y TIENE CAPACIDAD,EXTRACTO NO ENTREGADO,1,2011-04-29,13101168.0,2015-01-01 00:00:00,NaN,SIN PROFESION,SIN OFICIO


In [5]:
df.dropna(subset=['cuadrante', 'descripcion causal', 'ocupacion'], inplace=True)

# Dimensiones

In [6]:
print(f'Numero de muestras: {df.shape[0]}')
print(f'Variables: {df.shape[1]}\n')
df.info()

Numero de muestras: 87485
Variables: 27

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87485 entries, 0 to 89246
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   consecutivo documento deudor  87485 non-null  int64         
 1   nit                           87485 non-null  int64         
 2   nombres                       0 non-null      float64       
 3   numero documento              0 non-null      float64       
 4   tipo de producto              87485 non-null  int64         
 5   producto                      87485 non-null  object        
 6   valor obligacion              87485 non-null  float64       
 7   valor vencido                 87485 non-null  float64       
 8   regional                      87485 non-null  int64         
 9   region                        87485 non-null  object        
 10  abogado                       87485 non-null  int64  

# Valores Faltantes


In [7]:
df.isnull().mean().sort_values(ascending=False)

nombres                         1.000000
numero documento                1.000000
sector economico                0.567034
fecha ultima facturacion        0.100349
profesion                       0.005887
segmento                        0.000103
fecha traslado para cobro       0.000080
rango endeudamiento             0.000000
endeudamiento                   0.000000
fecha de perfeccionamiento      0.000000
calificacion real               0.000000
descripcion causal              0.000000
cuadrante                       0.000000
red                             0.000000
oficina radicacion              0.000000
consecutivo documento deudor    0.000000
nit                             0.000000
franja                          0.000000
dias mora                       0.000000
abogado                         0.000000
region                          0.000000
regional                        0.000000
valor vencido                   0.000000
valor obligacion                0.000000
producto        

In [8]:
# Downcasting
# Función para reducir el peso en memoria de un DataFrame
def downcast_dtypes(data:pd.DataFrame) -> pd.DataFrame:

    """
    Function to downcast any type variable

    Args:
        dataframe: DataFrame
    
    Return:
        DataFrame: Downcasted DataFrame
    """

    start = data.memory_usage(deep=True).sum() / 1024 ** 2
    float_cols = [col for col in data if data[col].dtype == 'float64']
    int_cols = [col for col in data if data[col].dtype in ['int64', 'int32']]
    object_cols = [col for col in data if data[col].dtype in ['object', 'bool']]

    data[float_cols] = data[float_cols].astype(np.float32)
    data[int_cols] = data[int_cols].astype(np.int16)
    data[object_cols] = data[object_cols].astype('category')

    end = data.memory_usage(deep=True).sum() / 1024 ** 2
    saved = (start - end) / start * 100
    print(f'Memory Saved: {saved:0.2f}%', '\n')

    return data.info()


In [9]:
downcast_dtypes(data=df)

Memory Saved: 91.41% 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87485 entries, 0 to 89246
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   consecutivo documento deudor  87485 non-null  int16         
 1   nit                           87485 non-null  int16         
 2   nombres                       0 non-null      float32       
 3   numero documento              0 non-null      float32       
 4   tipo de producto              87485 non-null  int16         
 5   producto                      87485 non-null  category      
 6   valor obligacion              87485 non-null  float32       
 7   valor vencido                 87485 non-null  float32       
 8   regional                      87485 non-null  int16         
 9   region                        87485 non-null  category      
 10  abogado                       87485 non-null  int16         
 11  dias 

In [10]:
# Capturar variables
# Función para capturar los tipos de variables
def capture_variables(data:pd.DataFrame) -> tuple:
    
    """
    Function to capture the types of Dataframe variables

    Args:
        dataframe: DataFrame
    
    Return:
        variables: A tuple of lists
    
    The order to unpack variables:
    1. numericals
    2. continous
    3. categoricals
    4. discretes
    5. temporaries
    """

    numericals = list(data.select_dtypes(include = [np.int16, np.float32]).columns)
    categoricals = list(data.select_dtypes(include = ['category', 'bool']).columns)
    temporaries = list(data.select_dtypes(include = ['datetime', 'timedelta']).columns)
    discretes = [col for col in data[numericals] if len(data[numericals][col].unique()) < 20]
    continuous = [col for col in data[numericals] if col not in discretes]

    # Variables
    print('\t\tTipos de variables')
    print(f'Hay {len(continuous)} variables continuas')
    print(f'Hay {len(discretes)} variables discretas')
    print(f'Hay {len(temporaries)} variables temporales')
    print(f'Hay {len(categoricals)} variables categóricas')
    
    variables = tuple((numericals, continuous, categoricals, discretes, temporaries))

    # Retornamos una tupla de listas
    return variables


In [11]:
numericals, continous, categorical, discretes, temporaries=  capture_variables(data=df)

		Tipos de variables
Hay 9 variables continuas
Hay 5 variables discretas
Hay 2 variables temporales
Hay 11 variables categóricas


In [12]:
# Uniformizar las variables categoricas
df[categorical] = df[categorical].applymap(lambda x: str(x).lower().strip())

In [13]:
# Valores faltantes
df.isnull().mean().sort_values(ascending=False)

nombres                         1.00000
numero documento                1.00000
fecha traslado para cobro       0.00008
segmento                        0.00000
profesion                       0.00000
sector economico                0.00000
rango endeudamiento             0.00000
endeudamiento                   0.00000
fecha de perfeccionamiento      0.00000
calificacion real               0.00000
descripcion causal              0.00000
cuadrante                       0.00000
red                             0.00000
oficina radicacion              0.00000
consecutivo documento deudor    0.00000
fecha ultima facturacion        0.00000
nit                             0.00000
franja                          0.00000
dias mora                       0.00000
abogado                         0.00000
region                          0.00000
regional                        0.00000
valor vencido                   0.00000
valor obligacion                0.00000
producto                        0.00000


In [14]:
df.columns

Index(['consecutivo documento deudor', 'nit', 'nombres', 'numero documento',
       'tipo de producto', 'producto', 'valor obligacion', 'valor vencido',
       'regional', 'region', 'abogado', 'dias mora', 'franja',
       'fecha traslado para cobro', 'fecha ultima facturacion', 'segmento',
       'oficina radicacion', 'red', 'cuadrante', 'descripcion causal',
       'calificacion real', 'fecha de perfeccionamiento', 'endeudamiento',
       'rango endeudamiento', 'sector economico', 'profesion', 'ocupacion'],
      dtype='object')

In [15]:
# list_var=['consecutivo documento deudor', 'nit', 'nombres', 'numero documento','tipo de producto',
#           'valor obligacion','regional','abogado','fecha traslado para cobro','fecha ultima facturacion',
#           'oficina radicacion', 'red','endeudamiento','sector economico', 'profesion','dias mora']
# df.drop(columns=list_var, inplace=True)

In [16]:
df.shape

(87485, 27)

In [17]:
df.columns

Index(['consecutivo documento deudor', 'nit', 'nombres', 'numero documento',
       'tipo de producto', 'producto', 'valor obligacion', 'valor vencido',
       'regional', 'region', 'abogado', 'dias mora', 'franja',
       'fecha traslado para cobro', 'fecha ultima facturacion', 'segmento',
       'oficina radicacion', 'red', 'cuadrante', 'descripcion causal',
       'calificacion real', 'fecha de perfeccionamiento', 'endeudamiento',
       'rango endeudamiento', 'sector economico', 'profesion', 'ocupacion'],
      dtype='object')

In [18]:
#%%
# =============================================================================
# FUNCIONES PARA ARREGLAR COLUMNAS Y CADENAS
# =============================================================================



#Función para arreglar los caracteres especiales de los encabezados
def fn_arregla_col(x):                 
    columnas=x.columns.values
   
    for i in range(len(columnas)):
        x.columns.values[i]=columnas[i].lower()
        x.columns.values[i]=columnas[i].strip()
        x.columns.values[i]=re.sub(' +','_', columnas[i])
        x.columns.values[i]=re.sub('\n','_', columnas[i])
        x.columns.values[i]=re.sub('á','a', columnas[i])
        x.columns.values[i]=re.sub('é','e', columnas[i])
        x.columns.values[i]=re.sub('í','i', columnas[i])
        x.columns.values[i]=re.sub('ó','o', columnas[i])
        x.columns.values[i]=re.sub('ú','u', columnas[i])
        x.columns.values[i]=re.sub('ñ','n', columnas[i])
        x.columns.values[i]=re.sub(':','', columnas[i])
        
    x.columns=columnas
    return x



#Función para arreglar los caracteres especiales de las colunmnas
def fn_arregla_cadena(x):
    
    if type(x)==np.nan:
        return x
    
    else:
    
        x=x.lower()
        x=re.sub('  +',' ', x)
        x=re.sub('á','a', x)
        x=re.sub('é','e', x)
        x=re.sub('í','i', x)
        x=re.sub('ó','o', x)
        x=re.sub('ú','u', x)
        x=re.sub('ñ','n', x)
        x=re.sub('nan','', x)
        x=re.sub('/','', x)
        x=re.sub('√ì','', x)
        x=re.sub('-','', x)
        
        x=x.strip().rstrip().lstrip()
        
    return x

In [19]:
df["descripcion causal"] = df["descripcion causal"].astype(str).apply(lambda x : fn_arregla_cadena(x))

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87485 entries, 0 to 89246
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   consecutivo documento deudor  87485 non-null  int16         
 1   nit                           87485 non-null  int16         
 2   nombres                       0 non-null      float32       
 3   numero documento              0 non-null      float32       
 4   tipo de producto              87485 non-null  int16         
 5   producto                      87485 non-null  object        
 6   valor obligacion              87485 non-null  float32       
 7   valor vencido                 87485 non-null  float32       
 8   regional                      87485 non-null  int16         
 9   region                        87485 non-null  object        
 10  abogado                       87485 non-null  int16         
 11  dias mora                   

In [21]:
df['fecha de perfeccionamiento'].value_counts()

2015-06-12    380
2014-08-04    188
2014-11-24    177
2014-12-23    175
2014-12-09    166
             ... 
2007-06-09      1
2007-11-03      1
1995-10-19      1
2014-01-01      1
1999-03-02      1
Name: fecha de perfeccionamiento, Length: 4360, dtype: int64

In [22]:
# Captura de datatime
time = df.columns[df.columns.str.contains('fecha','fec')]

# Transformar a variables temporales
df[time] =df[time].apply(lambda x: pd.to_datetime(x, infer_datetime_format=True))
df[time] =df[time].apply(lambda x: x.dt.strftime('%Y-%m-%d'))
df[time] =df[time].apply(lambda x: pd.to_datetime(x,infer_datetime_format=True))
df[time].sample(10)

ParserError: Unknown string format: 0001-01-01 bc present at position 1

In [ ]:
df['rango endeudamiento'].value_counts()

2015-01-01 00:00:00    44648
15-50                  18800
<1m                    18613
50-100                  4267
100-200                  909
>200                     248
Name: rango endeudamiento, dtype: int64

In [ ]:
df['rango endeudamiento'].replace('2015-01-01 00:00:00',"1-15", inplace=True)

In [ ]:
df['descripcion causal'].value_counts()

descuido                                             24546
sin causal de mora                                   20262
cuentas por cobrar                                    7778
calamidad                                             4059
retraso en pago de salario                            3664
ingresos insuficientes                                3613
disminucion de salario yo ingresos                    3347
desempleado titular yo cotitular                      3102
pago no aplicado                                      2910
no coincide fecha vencimiento con ingresos            2154
pago no aplicado o mal aplicado                       1988
sobreendeudamiento                                    1558
extracto no entregado                                 1361
cliente no informa causal                              978
pago otras deudas                                      871
viaje                                                  698
acostumbra a pagar atrasado                            6

# Save to CSV

In [ ]:
df.to_csv('cobranza.csv',index=False)

In [ ]:
df

,producto,valor vencido,region,franja,segmento,cuadrante,descripcion causal,calificacion real,fecha de perfeccionamiento,rango endeudamiento,ocupacion
0,tarjetas de credito,152093.0,bogota,<30,personal plus,tiene voluntad y tiene capacidad reducida,calamidad,2,1978-12-06,1-15,sin oficio
1,consumo,40584.0,bogota,91-120,personal plus,sin cuadrante,sin causal de mora,3,2015-03-11,<1m,sin oficio
2,tarjetas de credito,547088.0,sur,31-60,personas,tiene voluntad y tiene capacidad reducida,retraso en pago de salario,1,2003-03-27,1-15,jubilado
3,tarjetas de credito,1455631.0,bogota,61-90,pymes,no tiene voluntad y tiene capacidad,descuido,1,1990-09-01,1-15,sin oficio
4,consumo,2221125.0,centro,31-60,personal plus,tiene voluntad y tiene capacidad,descuido,2,2015-05-12,1-15,taxista
...,...,...,...,...,...,...,...,...,...,...,...
89242,consumo,284377.0,sur,91-120,personal plus,tiene voluntad y tiene capacidad reducida,descuido,3,2013-05-27,15-50,empleado sector publico
89243,consumo,2790040.0,sur,91-120,personal plus,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2013-03-18,15-50,sin oficio
89244,consumo,1511822.0,sur,91-120,personal plus,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2014-06-03,15-50,sin oficio
89245,consumo,609495.0,sur,91-120,personal plus,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2014-09-29,15-50,sin oficio


In [ ]:
df.isnull().sum()

producto                      0
valor vencido                 0
region                        0
franja                        0
segmento                      0
cuadrante                     0
descripcion causal            0
calificacion real             0
fecha de perfeccionamiento    0
rango endeudamiento           0
ocupacion                     0
dtype: int64

In [23]:
df

,consecutivo documento deudor,nit,nombres,numero documento,tipo de producto,producto,valor obligacion,valor vencido,regional,region,...,red,cuadrante,descripcion causal,calificacion real,fecha de perfeccionamiento,endeudamiento,rango endeudamiento,sector economico,profesion,ocupacion
0,-2112,18,NaN,NaN,3,tarjetas de credito,5637605.0,152093.0,2099,bogota,...,1,tiene voluntad y tiene capacidad reducida,calamidad,2,1978-12-06,5637605.0,2015-01-01 00:00:00,nan,sin profesion,sin oficio
1,25068,22,NaN,NaN,5,consumo,40584.0,40584.0,2099,bogota,...,1,sin cuadrante,sin causal de mora,3,2015-03-11,40584.0,<1m,nan,sin profesion,sin oficio
2,21885,23,NaN,NaN,3,tarjetas de credito,2786178.0,547088.0,3099,sur,...,1,tiene voluntad y tiene capacidad reducida,retraso en pago de salario,1,2003-03-27,7747280.0,2015-01-01 00:00:00,nan,sin profesion,jubilado
3,18037,46,NaN,NaN,3,tarjetas de credito,5904653.0,1455631.0,2099,bogota,...,1,no tiene voluntad y tiene capacidad,descuido,1,1990-09-01,5904653.0,2015-01-01 00:00:00,nan,sin profesion,sin oficio
4,25406,92,NaN,NaN,5,consumo,2221125.0,2221125.0,6399,centro,...,1,tiene voluntad y tiene capacidad,descuido,2,2015-05-12,2221125.0,2015-01-01 00:00:00,agropecuario,agricultor,taxista
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89242,-11228,1025,NaN,NaN,89,consumo,2048599.0,284377.0,3099,sur,...,1,tiene voluntad y tiene capacidad reducida,descuido,3,2013-05-27,37088352.0,15-50,educacion,profesor,empleado sector publico
89243,-25346,1027,NaN,NaN,4,consumo,33835904.0,2790040.0,3099,sur,...,1,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2013-03-18,45757680.0,15-50,nan,sin profesion,sin oficio
89244,-29395,1027,NaN,NaN,88,consumo,1540300.0,1511822.0,3099,sur,...,1,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2014-06-03,45757680.0,15-50,nan,sin profesion,sin oficio
89245,-25794,1027,NaN,NaN,88,consumo,3954718.0,609495.0,3099,sur,...,1,no tiene voluntad y tiene capacidad,desempleado titular yo cotitular,3,2014-09-29,45757680.0,15-50,nan,sin profesion,sin oficio
